# Pull the data from Athena and turn it into training data

Create a series of temperature strings that look like this:

[t0, t1 ... t23]

...with 24 temperatures for a given site.  Create these by sliding a window along unique temperatures by site.

TODO:

It's not actually learning...

More than one site
Deal with nulls
Better windowing: https://www.tensorflow.org/tutorials/structured_data/time_series#4_create_tfdatadatasets
Other data as inputs (pressure, wind speed etc)
Encode time as input (e.g. hour of day)
Longer input history
More sites in input
Normalise the data (temp => -1 - 1 etc)
Wind speed and direction as vector (snazzy!)


https://www.tensorflow.org/tutorials/structured_data/time_series

In [88]:
import pandas as pd
import numpy as np
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

cursor = connect(s3_staging_dir="s3://dantelore.queryresults/pyathena/",
                 region_name="eu-west-1", cursor_class=PandasCursor).cursor()


length = 2
sub_sequences = []

sites_df = pd.DataFrame(['WITTERING'], columns=['site_name']) #cursor.execute("select distinct(site_name) from lake.weather").as_pandas()
for site_name in sites_df['site_name']:
    df = cursor.execute(f"select observation_ts, temperature from lake.weather where site_name = '{site_name}' order by observation_ts asc").as_pandas()

    df['temperature'] = df['temperature'].astype('float32')

    train_mean = df['temperature'].mean()
    train_std = df['temperature'].std()

    df['temperature'] = (df['temperature'] - train_mean) / train_std

    all_temps = df['temperature'].values

    for i in range(0, len(all_temps) - length):
        sub_sequences.append(all_temps[i:i+length])

input_sequences = np.array(sub_sequences)

xs = input_sequences[:,:-1]
ys = input_sequences[:,-1]

[(x * train_std) + train_mean for x in xs[0]]


[11.1]

In [104]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(units=1, activation='relu', input_dim=length - 1)
])

adam = Adam(lr=0.1)
model.compile(loss='mse', optimizer=adam, metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


/Users/dan/Development/simple-prediction/.venv/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [105]:
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import clear_output

history_df = pd.DataFrame(columns=['loss', 'accuracy'])

run_count = 1
epochs_per_run = 100

for run in range(0, run_count):
    print(f"Starting Run {run}/{run_count}")
    # Use verbose=2 here to prevent progreess bars locking up jupyter after a few hours
    history = model.fit(xs, ys, epochs=epochs_per_run, verbose=2)
    
    dt = datetime.now() 
    model_filename = f"data/weather_models/model_{dt.year}_{dt.month}_{dt.day}_{dt.hour}_{dt.minute}.h5"
    #model.save(model_filename)

    history_df = pd.concat([history_df, pd.DataFrame(history.history)], ignore_index=True)

    plt.plot(history_df['accuracy'])
    plt.title(f'Model Training Progress - Run {run} of {run_count}')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.show()

Starting Run 0/1
Epoch 1/100


2023-01-23 00:12:14.429369: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


292/292 - 1s - loss: 0.5402 - mean_absolute_error: 0.5146 - 1s/epoch - 4ms/step
Epoch 2/100
292/292 - 1s - loss: 0.4666 - mean_absolute_error: 0.4613 - 1s/epoch - 4ms/step
Epoch 3/100
292/292 - 1s - loss: 0.4668 - mean_absolute_error: 0.4615 - 978ms/epoch - 3ms/step
Epoch 4/100
292/292 - 1s - loss: 0.4673 - mean_absolute_error: 0.4630 - 975ms/epoch - 3ms/step
Epoch 5/100
292/292 - 1s - loss: 0.4669 - mean_absolute_error: 0.4622 - 984ms/epoch - 3ms/step
Epoch 6/100
292/292 - 1s - loss: 0.4673 - mean_absolute_error: 0.4632 - 975ms/epoch - 3ms/step
Epoch 7/100
292/292 - 1s - loss: 0.4678 - mean_absolute_error: 0.4639 - 969ms/epoch - 3ms/step
Epoch 8/100
292/292 - 1s - loss: 0.4678 - mean_absolute_error: 0.4641 - 1s/epoch - 4ms/step
Epoch 9/100
292/292 - 1s - loss: 0.4673 - mean_absolute_error: 0.4629 - 977ms/epoch - 3ms/step
Epoch 10/100
292/292 - 1s - loss: 0.4673 - mean_absolute_error: 0.4628 - 981ms/epoch - 3ms/step
Epoch 11/100
292/292 - 1s - loss: 0.4672 - mean_absolute_error: 0.4628